### **Merging rxnorm to dti dataset**


In [1]:
%pip install PyYAML
%pip isntall pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "isntall" - maybe you meant "install"



In [2]:
import pandas as pd
import yaml

In [3]:
# Load the YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [4]:
# pd load
scope_pd = pd.read_parquet(config["paths"]["LINK_TO_DTI_DATASET"])


In [5]:
scope_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208221 entries, 0 to 2208220
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   drug_chembl_id     object 
 1   target_uniprot_id  object 
 2   positive_times     int64  
 3   negative_times     int64  
 4   drug_pubchem_id    float64
 5   label              int64  
 6   smiles             object 
 7   sequence           object 
 8   molfile_3d         object 
dtypes: float64(1), int64(3), object(5)
memory usage: 151.6+ MB


In [8]:
scope_subset = scope_pd[
    [
        "drug_chembl_id",
        "target_uniprot_id",
        "label",
        "smiles",
        "sequence",
        "molfile_3d",
    ]
]


In [9]:
scope_subset.to_parquet("scope_subset.parquet", index=False)